In [3]:
import pandas as pd
import os
import sqlite3

In [4]:
os.makedirs("raw", exist_ok=True)
os.makedirs("processed", exist_ok=True)

In [9]:
import os
os.listdir("raw")

[]

In [11]:
users = pd.read_csv("raw/users.csv")
orders = pd.read_csv("raw/orders.csv")
order_items = pd.read_csv("raw/order_items.csv")
products = pd.read_csv("raw/products.csv")

In [12]:
print(users.shape, orders.shape, order_items.shape, products.shape)

(10000, 6) (20000, 5) (43525, 7) (2000, 6)


In [13]:
for name, df in {
    "users": users,
    "orders": orders,
    "order_items": order_items,
    "products": products
}.items():
    print(f"\n{name.upper()}")
    print(df.info())
    print(df.isnull().sum())


USERS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      10000 non-null  object
 1   name         10000 non-null  object
 2   email        10000 non-null  object
 3   gender       10000 non-null  object
 4   city         10000 non-null  object
 5   signup_date  10000 non-null  object
dtypes: object(6)
memory usage: 468.9+ KB
None
user_id        0
name           0
email          0
gender         0
city           0
signup_date    0
dtype: int64

ORDERS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      20000 non-null  object 
 1   user_id       20000 non-null  object 
 2   order_date    20000 non-null  object 
 3   order_status  20000 non-null  object 
 4   total_amount  200

In [14]:
users.drop_duplicates(inplace=True)
orders.drop_duplicates(inplace=True)
order_items.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)

In [15]:
def clean_columns(df):
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    return df

users = clean_columns(users)
orders = clean_columns(orders)
order_items = clean_columns(order_items)
products = clean_columns(products)

In [16]:
if 'created_at' in orders.columns:
    orders['created_at'] = pd.to_datetime(orders['created_at'])

if 'created_at' in users.columns:
    users['created_at'] = pd.to_datetime(users['created_at'])

In [17]:
order_value = (
    order_items
    .groupby('order_id')['sale_price']
    .sum()
    .reset_index()
    .rename(columns={'sale_price': 'total_order_value'})
)

KeyError: 'Column not found: sale_price'

In [19]:
type(order_items)

pandas.core.frame.DataFrame

In [20]:
order_items.dropna(inplace=True)

In [22]:
import pandas as pd

order_items = pd.read_csv("raw/order_items.csv")

In [23]:
order_items.dropna(inplace=True)

In [24]:
print(order_items.columns)

Index(['order_item_id', 'order_id', 'product_id', 'user_id', 'quantity',
       'item_price', 'item_total'],
      dtype='object')


In [25]:
order_value = (
    order_items
    .groupby('order_id')['item_total']
    .sum()
    .reset_index()
    .rename(columns={'item_total': 'total_order_value'})
)

In [27]:
order_value.head()

,order_id,total_order_value
0,O00000001,689.66
1,O00000002,1666.85
2,O00000003,665.06
3,O00000004,689.50
4,O00000005,860.50


In [28]:
orders = pd.read_csv("raw/orders.csv")

In [30]:
print(orders.columns)

Index(['order_id', 'user_id', 'order_date', 'order_status', 'total_amount'], dtype='object')


In [31]:
orders = orders.merge(
    order_value,
    on='order_id',
    how='left'
)

In [32]:
orders[['order_id', 'total_order_value']].head()

,order_id,total_order_value
0,O00000001,689.66
1,O00000002,1666.85
2,O00000003,665.06
3,O00000004,689.50
4,O00000005,860.50


In [34]:
orders['total_order_value'] = orders['total_order_value'].fillna(0)

In [35]:
orders.head()

,order_id,user_id,order_date,order_status,total_amount,total_order_value
0,O00000001,U009310,2025-09-09T14:52:37.292731,processing,689.66,689.66
1,O00000002,U003247,2025-04-15T01:18:27.193404,completed,1666.85,1666.85
2,O00000003,U007252,2025-04-27T15:37:48.008624,processing,665.06,665.06
3,O00000004,U008986,2025-10-04T20:35:22.204857,cancelled,689.50,689.50
4,O00000005,U008537,2024-11-13T08:15:18.498252,cancelled,860.50,860.50


In [36]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           20000 non-null  object 
 1   user_id            20000 non-null  object 
 2   order_date         20000 non-null  object 
 3   order_status       20000 non-null  object 
 4   total_amount       20000 non-null  float64
 5   total_order_value  20000 non-null  float64
dtypes: float64(2), object(4)
memory usage: 937.6+ KB


In [38]:
print(orders.columns)

Index(['order_id', 'user_id', 'order_date', 'order_status', 'total_amount',
       'total_order_value'],
      dtype='object')


In [39]:
orders['order_date'] = pd.to_datetime(orders['order_date'], errors='coerce')

In [40]:
orders = orders.drop_duplicates(subset='order_id')

In [41]:
orders.isna().sum()

,0
order_id,0
user_id,0
order_date,0
order_status,0
total_amount,0
total_order_value,0


In [42]:
orders['total_order_value'] = orders['total_order_value'].fillna(0)
orders['order_status'] = orders['order_status'].fillna('unknown')

In [43]:
orders['order_year'] = orders['order_date'].dt.year
orders['order_month'] = orders['order_date'].dt.month
orders['order_day'] = orders['order_date'].dt.day

In [44]:
orders[['order_id', 'order_date', 'order_year', 'order_month', 'total_order_value']].head()

,order_id,order_date,order_year,order_month,total_order_value
0,O00000001,2025-09-09 14:52:37.292731,2025,9,689.66
1,O00000002,2025-04-15 01:18:27.193404,2025,4,1666.85
2,O00000003,2025-04-27 15:37:48.008624,2025,4,665.06
3,O00000004,2025-10-04 20:35:22.204857,2025,10,689.50
4,O00000005,2024-11-13 08:15:18.498252,2024,11,860.50


In [45]:
users.head()

,user_id,name,email,gender,city,signup_date
0,U000001,Angel Hill,donaldgarcia@example.net,Other,New Roberttown,2025-03-13
1,U000002,Jesse Guzman,jennifermiles@example.com,Male,South Bridget,2024-03-05
2,U000003,Adam Shaffer,jpeterson@example.org,Male,Curtisfurt,2025-07-07
3,U000004,Melanie Munoz,blairamanda@example.com,Other,New Kellystad,2024-03-07
4,U000005,Janet Williams,kendragalloway@example.org,Female,South Joshuastad,2025-01-29


In [46]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      10000 non-null  object
 1   name         10000 non-null  object
 2   email        10000 non-null  object
 3   gender       10000 non-null  object
 4   city         10000 non-null  object
 5   signup_date  10000 non-null  object
dtypes: object(6)
memory usage: 468.9+ KB


In [47]:
print(users.columns)

Index(['user_id', 'name', 'email', 'gender', 'city', 'signup_date'], dtype='object')


In [48]:
users = users.drop_duplicates(subset='user_id')

In [49]:
users.isna().sum()

,0
user_id,0
name,0
email,0
gender,0
city,0
signup_date,0


In [50]:
users['gender'] = users['gender'].fillna('unknown')
users['city'] = users['city'].fillna('unknown')

In [51]:
users.head()

,user_id,name,email,gender,city,signup_date
0,U000001,Angel Hill,donaldgarcia@example.net,Other,New Roberttown,2025-03-13
1,U000002,Jesse Guzman,jennifermiles@example.com,Male,South Bridget,2024-03-05
2,U000003,Adam Shaffer,jpeterson@example.org,Male,Curtisfurt,2025-07-07
3,U000004,Melanie Munoz,blairamanda@example.com,Other,New Kellystad,2024-03-07
4,U000005,Janet Williams,kendragalloway@example.org,Female,South Joshuastad,2025-01-29


In [52]:
print(orders.columns)
print(users.columns)

Index(['order_id', 'user_id', 'order_date', 'order_status', 'total_amount',
       'total_order_value', 'order_year', 'order_month', 'order_day'],
      dtype='object')
Index(['user_id', 'name', 'email', 'gender', 'city', 'signup_date'], dtype='object')


In [53]:
orders_users = orders.merge(
    users,
    on='user_id',
    how='left'
)

In [54]:
orders_users.head()

,order_id,user_id,order_date,order_status,total_amount,total_order_value,order_year,order_month,order_day,name,email,gender,city,signup_date
0,O00000001,U009310,2025-09-09 14:52:37.292731,processing,689.66,689.66,2025,9,9,Christine Snyder,john14@example.com,Female,Jonestown,2024-02-22
1,O00000002,U003247,2025-04-15 01:18:27.193404,completed,1666.85,1666.85,2025,4,15,Ryan Valencia,stevensonanthony@example.net,Male,South Mary,2024-03-23
2,O00000003,U007252,2025-04-27 15:37:48.008624,processing,665.06,665.06,2025,4,27,Tammy Fox,roberthenderson@example.org,Other,Wilsonton,2024-06-27
3,O00000004,U008986,2025-10-04 20:35:22.204857,cancelled,689.50,689.50,2025,10,4,Kurt Howard,nancy40@example.org,Female,New Barbarafurt,2024-12-04
4,O00000005,U008537,2024-11-13 08:15:18.498252,cancelled,860.50,860.50,2024,11,13,Kevin Krause,kyliechandler@example.net,Male,New Christophermouth,2025-03-12


In [55]:
orders_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   order_id           20000 non-null  object        
 1   user_id            20000 non-null  object        
 2   order_date         20000 non-null  datetime64[ns]
 3   order_status       20000 non-null  object        
 4   total_amount       20000 non-null  float64       
 5   total_order_value  20000 non-null  float64       
 6   order_year         20000 non-null  int32         
 7   order_month        20000 non-null  int32         
 8   order_day          20000 non-null  int32         
 9   name               20000 non-null  object        
 10  email              20000 non-null  object        
 11  gender             20000 non-null  object        
 12  city               20000 non-null  object        
 13  signup_date        20000 non-null  object        
dtypes: dat

In [56]:
orders_users['user_id'].isna().sum()

np.int64(0)

In [57]:
orders_users.to_csv(
    'processed/orders_users.csv',
    index=False
)

In [58]:
total_revenue = orders['total_order_value'].sum()
total_revenue

np.float64(11918668.950000001)

In [59]:
total_orders = orders['order_id'].nunique()
total_orders

20000

In [60]:
total_customers = orders['user_id'].nunique()
total_customers

8635

In [61]:
average_order_value = total_revenue / total_orders
average_order_value

np.float64(595.9334475)

In [62]:
orders_per_customer = total_orders / total_customers
orders_per_customer

2.3161551823972206

In [63]:
kpi_summary = pd.DataFrame({
    'Metric': [
        'Total Revenue',
        'Total Orders',
        'Total Customers',
        'Average Order Value',
        'Orders per Customer'
    ],
    'Value': [
        total_revenue,
        total_orders,
        total_customers,
        average_order_value,
        orders_per_customer
    ]
})

kpi_summary

,Metric,Value
0,Total Revenue,1.191867e+07
1,Total Orders,2.000000e+04
2,Total Customers,8.635000e+03
3,Average Order Value,5.959334e+02
4,Orders per Customer,2.316155e+00


In [64]:
kpi_summary.to_csv(
    'processed/kpi_summary.csv',
    index=False
)

In [65]:
orders['order_date'] = pd.to_datetime(orders['order_date'], errors='coerce')

revenue_by_date = (
    orders
    .groupby(orders['order_date'].dt.to_period('M'))['total_order_value']
    .sum()
    .reset_index()
)

revenue_by_date['order_date'] = revenue_by_date['order_date'].astype(str)
revenue_by_date

,order_date,total_order_value
0,2024-01,543120.43
1,2024-02,513712.79
2,2024-03,541213.26
3,2024-04,545624.46
4,2024-05,552700.82
5,2024-06,548534.32
6,2024-07,587135.76
7,2024-08,519474.77
8,2024-09,529067.12
9,2024-10,529215.48


In [66]:
revenue_by_date.to_csv(
    'processed/revenue_by_month.csv',
    index=False
)

In [67]:
order_status_dist = (
    orders
    .groupby('order_status')['order_id']
    .count()
    .reset_index(name='order_count')
)

order_status_dist

,order_status,order_count
0,cancelled,3920
1,completed,4021
2,processing,3880
3,returned,4066
4,shipped,4113


In [68]:
order_status_dist.to_csv(
    'processed/order_status_dist.csv',
    index=False
)

In [69]:
top_customers = (
    orders
    .groupby('user_id')['total_order_value']
    .sum()
    .reset_index()
    .sort_values(by='total_order_value', ascending=False)
    .head(10)
)

top_customers

,user_id,total_order_value
8571,U009931,13288.88
5374,U006233,11234.22
5572,U006469,10953.18
7222,U008370,10190.11
4918,U005702,10175.81
8549,U009903,10118.31
5,U000006,9952.28
4238,U004906,9538.69
5372,U006231,9486.29
6834,U007930,8947.98


In [70]:
top_customers.to_csv(
    'processed/top_customers.csv',
    index=False
)

In [71]:
orders_per_user = (
    orders
    .groupby('user_id')['order_id']
    .count()
    .reset_index(name='order_count')
)

orders_per_user

,user_id,order_count
0,U000001,1
1,U000002,4
2,U000003,3
3,U000004,3
4,U000005,1
...,...,...
8630,U009996,1
8631,U009997,2
8632,U009998,2
8633,U009999,1


In [72]:
orders_per_user.to_csv(
    'processed/orders_per_user.csv',
    index=False
)

In [73]:
orders.to_csv('processed/orders_final.csv', index=False)
users.to_csv('processed/users_final.csv', index=False)

In [74]:
import sqlite3

conn = sqlite3.connect("ecommerce_etl.sqlite")

In [75]:
customers_clean.to_sql(
    "customers",
    conn,
    if_exists="replace",
    index=False
)

orders.to_sql(
    "orders",
    conn,
    if_exists="replace",
    index=False
)

products_clean.to_sql(
    "products",
    conn,
    if_exists="replace",
    index=False
)

NameError: name 'customers_clean' is not defined

In [76]:
customers_clean = users[
    ['user_id', 'gender', 'city', 'country']
].copy()

KeyError: "['country'] not in index"

In [77]:
print(users.columns)

Index(['user_id', 'name', 'email', 'gender', 'city', 'signup_date'], dtype='object')


In [78]:
customers_clean = users[
    ['user_id', 'gender', 'city']
].copy()

In [79]:
customers_clean.head()

,user_id,gender,city
0,U000001,Other,New Roberttown
1,U000002,Male,South Bridget
2,U000003,Male,Curtisfurt
3,U000004,Other,New Kellystad
4,U000005,Female,South Joshuastad


In [80]:
import sqlite3

conn = sqlite3.connect("ecommerce_etl.sqlite")

customers_clean.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
products_clean.to_sql("products", conn, if_exists="replace", index=False)

conn.close()

NameError: name 'products_clean' is not defined

In [81]:
print(products.columns)

Index(['product_id', 'product_name', 'category', 'brand', 'price', 'rating'], dtype='object')


In [82]:
products_clean = products[
    ['product_id', 'product_name', 'category', 'price']
].copy()

In [83]:
products_clean.head()

,product_id,product_name,category,price
0,P000001,Astra Be,Clothing,157.89
1,P000002,NeoTech Someone,Groceries,21.46
2,P000003,Acme Discuss,Sports,265.37
3,P000004,Nimbus South,Electronics,541.41
4,P000005,Astra Capital,Home & Kitchen,198.00


In [84]:
import sqlite3

conn = sqlite3.connect("ecommerce_etl.sqlite")

customers_clean.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
products_clean.to_sql("products", conn, if_exists="replace", index=False)

conn.close()

In [85]:
import sqlite3

conn = sqlite3.connect("ecommerce_etl.sqlite")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

conn.close()